In [1]:
# inception v3,v4の実装
# めんどうなので一旦v2

In [2]:
# 10章
# CNNの基礎


# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import time
%matplotlib inline



# Training settings
batch_size = 64



In [ ]:
# MNIST Dataset
# train_dataset = datasets.MNIST(root='./data/',
#                                train=True,
#                                transform=transforms.ToTensor(),
#                                download=True)

# test_dataset = datasets.MNIST(root='./data/',
#                               train=False,
#                               transform=transforms.ToTensor())

# # Data Loader (Input Pipeline)
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
#                                            batch_size=batch_size,
#                                            shuffle=True)

# test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
#                                           batch_size=batch_size,
#                                           shuffle=False)



In [ ]:
# v3の全貌

#   conv0             | Conv2d_1a_3x3
#   conv1             | Conv2d_2a_3x3
#   conv2             | Conv2d_2b_3x3
#   pool1             | MaxPool_3a_3x3
#   conv3             | Conv2d_3b_1x1
#   conv4             | Conv2d_4a_3x3
#   pool2             | MaxPool_5a_3x3
#   mixed_35x35x256a  | Mixed_5b
#   mixed_35x35x288a  | Mixed_5c
#   mixed_35x35x288b  | Mixed_5d
#   mixed_17x17x768a  | Mixed_6a
#   mixed_17x17x768b  | Mixed_6b
#   mixed_17x17x768c  | Mixed_6c
#   mixed_17x17x768d  | Mixed_6d
#   mixed_17x17x768e  | Mixed_6e
#   mixed_8x8x1280a   | Mixed_7a
#   mixed_8x8x2048a   | Mixed_7b
#   mixed_8x8x2048b   | Mixed_7c
# このあとaverage, dropout, 全結合, softmaxとつづく


# v2全貌
# 種類         / patch,stride  / input_size
# conv         /     3*3/2     / 299*299*3
# conv         /     3*3/1     / 149*149*32
# conv padded  /     3*3/1     / 147*147*32
# pool         /     3*3/2     / 147*147*64
# conv         /     3*3/1     / 73*73*64
# conv         /     3*3/2     / 71*71*80
# conv         /     3*3/1     / 35*35*192
# inception3   /     別途      / 35*35*288
# inception5   /     別途      / 17*17*768
# inception2   /     別途      / 8*8*1280
# pool         /     8*8       / 8*8*2048
# linear       /     logit     / 1*1*2048
# softmax      /     classifier/ 1*1*1000  



In [ ]:
# インセプションモデルの実装
# 種類         / patch,stride  / input_size
# inception3   /     別途      / 35*35*288
# inception5   /     別途      / 17*17*768
# inception2   /     別途      / 8*8*1280

class Inception3(nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)




In [ ]:
# ネットワークの設計
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)




In [ ]:
# トレーニングとテストの内容
model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            tr.append(loss.data[0])
    return tr


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    ts.append(test_loss)
    return ts



In [ ]:
# 実行部分
tr, ts = [], []
start = time.time()
for epoch in range(1, 10):
    t = train(epoch)
    s = test()
    tr.append(t)
    ts.append(s)
end = time.time()

print("taken_time is:", end-start," sec")